In [ ]:
from new_main import parse_config, instantiate_agents, instantiate_auction, instantiate_publishers
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from collections import defaultdict

In [ ]:
# Parse configuration file
(rng, config, agent_configs, agents2items, agents2item_values, publisher_configs, num_runs, max_slots) = parse_config('../new_config/FP_Oracle.json')

In [ ]:
def run_repeated_auctions():
    agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)
    auction, num_iter, rounds_per_iter, output_dir = instantiate_auction(rng, config, agents2items, agents2item_values, agents, max_slots)
    publishers = instantiate_publishers(rng, publisher_configs)
    
    agent2net_utility = defaultdict(list)
    agent2gross_utility = defaultdict(list)
    
    auction_revenue = []
    
    for i in range(num_iter):
        print(f'==== ITERATION {i} ====')
        # Round-robin over publishers
        pub_auctions = {publisher.name: publisher.num_auctions for publisher in publishers}
        total_auctions = sum(pub_auctions.values())

        with tqdm(total=total_auctions) as pbar:
            while sum(pub_auctions.values()) > 0:
                for publisher in publishers:
                    if pub_auctions[publisher.name] > 0:
                        # Generate a user from the current publisher
                        curr_user_context = publisher.generate_user_context()
                        # Simulate the auction
                        auction.simulate_opportunity(curr_user_context, publisher.name)
                        pub_auctions[publisher.name] -= 1
                        pbar.update(1)

        for agent_id, agent in enumerate(auction.agents):
            print(f'Agent {agent.name} has spent {agent.get_agent_spent()}, '
                  f'winning {agent.get_agent_opp_won()} opportunities (win ratio: {agent.get_agent_won_ratio()})')
            print(f'Mean value paid for a won bid: {agent.get_mean_won_bid()}')

            agent.update(iteration=i)

            agent2net_utility[agent.name].append(agent.net_utility)
            agent2gross_utility[agent.name].append(agent.gross_utility)

            agent.clear_utility()
            agent.clear_logs()

        auction_revenue.append(auction.revenue)
        auction.clear_revenue()


In [ ]:
def sample_fixed_publisher(chosen_publisher_list, num_auctions=10000):